In [72]:
import os

from keybert import KeyBERT
import whisper
import urllib.request
from pydub import AudioSegment
from pydub.silence import split_on_silence
import math

In [2]:
# Configurations

# anything under SILENT_TRESH dBFS is considered silence
# if clips duration are so long, SILENT_TRESH must be a lower value (between 60 an 85 must be ok)
SILENT_TRESH = 60
MAX_LOCAL_TOPICS = 4
LANGUAGE = "English"


# AUDIO_URL = "https://stakwork-uploads-dev.s3.amazonaws.com/short_ep_48_kevin_rooke.mp3"
AUDIO_URL = "https://soundcloud.com/bitcoin-podcast/corb-lund-the-bitcoin-cowboy?si=aea5659804774d60827f98bd5f4db427&utm_source=clipboard&utm_medium=text&utm_campaign=social_sharing"

filebasename = "audio"

#inits
model = whisper.load_model("base")
kw_model = KeyBERT()

# Download audio file
urllib.request.urlretrieve(AUDIO_URL, filebasename + ".mp3")

# Convert audio file
# sound = AudioSegment.from_mp3(filebasename + ".mp3")
sound = AudioSegment.from_mp3("btccowboy" + ".mp3")
sound.export(filebasename + ".wav", format="wav")


<_io.BufferedRandom name='audio.wav'>

In [4]:
# Split the audio in silents and put them in a list called audio_chunks


#remove old chunks in folder
mydir="./"
for f in os.listdir(mydir):
    if not f.startswith("chunk"):
        continue
    os.remove(f)

#Splittin audio in silences
finalList = []
print("THIS PROCESS COULD TAKE LONG, PLEASE BE PATIENT!")
audio_chunks = split_on_silence(sound, silence_thresh=-SILENT_TRESH)
print("audio_chunks:",audio_chunks)
print("DONE!")

THIS PROCESS COULD TAKE LONG, PLEASE BE PATIENT!
audio_chunks: [<pydub.audio_segment.AudioSegment object at 0x296048760>, <pydub.audio_segment.AudioSegment object at 0x296048af0>, <pydub.audio_segment.AudioSegment object at 0x2960541f0>, <pydub.audio_segment.AudioSegment object at 0x296054be0>, <pydub.audio_segment.AudioSegment object at 0x2960542e0>, <pydub.audio_segment.AudioSegment object at 0x296054970>, <pydub.audio_segment.AudioSegment object at 0x2960546a0>, <pydub.audio_segment.AudioSegment object at 0x296054130>, <pydub.audio_segment.AudioSegment object at 0x1040f96d0>, <pydub.audio_segment.AudioSegment object at 0x1040f9880>, <pydub.audio_segment.AudioSegment object at 0x1040f9970>, <pydub.audio_segment.AudioSegment object at 0x2ae90bdc0>, <pydub.audio_segment.AudioSegment object at 0x2ae90bf70>, <pydub.audio_segment.AudioSegment object at 0x2ae90be50>, <pydub.audio_segment.AudioSegment object at 0x2ae90bfd0>, <pydub.audio_segment.AudioSegment object at 0x2ae90bd90>, <pydub.a

In [85]:
# get the audio_chunks list and calculate the timestamps. Then save them in a timestamps list

totalduration = 0
for chunk in audio_chunks:
    totalduration += chunk.duration_seconds
print("original sound duration:", sound.duration_seconds)
print("totalduraton:", totalduration)

timestamps=[]
def get_start_end_time(audio_chunks):
    #get the start time and the end time of each chunk
    start_time = 0
    end_time = 0
    for i, chunk in enumerate(audio_chunks):
        end_time = start_time + len(chunk)
        timestamps.append([i,math.trunc(start_time/1000),math.trunc(end_time/1000)])
        print("chunk",i,":",math.trunc(start_time/1000),"-",math.trunc(end_time/1000) , " ----- ",math.trunc(chunk.duration_seconds))

        start_time = end_time
    return

get_start_end_time(audio_chunks)
print(timestamps)

original sound duration: 2065.031836734694
totalduraton: 2012.0889795918372
chunk 0 : 0 - 80  -----  80
chunk 1 : 80 - 97  -----  17
chunk 2 : 97 - 191  -----  93
chunk 3 : 191 - 252  -----  61
chunk 4 : 252 - 266  -----  14
chunk 5 : 266 - 423  -----  156
chunk 6 : 423 - 429  -----  6
chunk 7 : 429 - 435  -----  5
chunk 8 : 435 - 475  -----  39
chunk 9 : 475 - 561  -----  86
chunk 10 : 561 - 708  -----  146
chunk 11 : 708 - 728  -----  20
chunk 12 : 728 - 814  -----  85
chunk 13 : 814 - 918  -----  104
chunk 14 : 918 - 984  -----  65
chunk 15 : 984 - 996  -----  12
chunk 16 : 996 - 1059  -----  63
chunk 17 : 1059 - 1075  -----  15
chunk 18 : 1075 - 1108  -----  32
chunk 19 : 1108 - 1161  -----  53
chunk 20 : 1161 - 1174  -----  12
chunk 21 : 1174 - 1228  -----  54
chunk 22 : 1228 - 1237  -----  8
chunk 23 : 1237 - 1278  -----  40
chunk 24 : 1278 - 1375  -----  97
chunk 25 : 1375 - 1381  -----  5
chunk 26 : 1381 - 1500  -----  119
chunk 27 : 1500 - 1503  -----  2
chunk 28 : 1503 - 1534

In [86]:
# CAN BE DELETED
# Print results to check duration
totalduration = 0
for chunk in audio_chunks:
    totalduration += chunk.duration_seconds
print("original sound duration:", sound.duration_seconds)
print("totalduraton:", totalduration)

original sound duration: 2065.031836734694
totalduraton: 2012.0889795918372


In [87]:
# get the topics of each chunk in audio_chunk list and obtain a list with order, chunk, topics othe chunk and timestamps called chunk_topic_list

chunk_topic_list=[]
for i, chunk in enumerate(audio_chunks):
   out_file = "chunk{0}.wav".format(i)
   chunk.export(out_file, format="wav")
   transcriptLocal = model.transcribe(out_file, fp16=False, max_initial_timestamp=None, verbose=False, language=LANGUAGE)
   keywordsLocal = kw_model.extract_keywords(transcriptLocal["text"], top_n=MAX_LOCAL_TOPICS)
   local_topics=[]
   for j in range(len(keywordsLocal)):
       if j <= MAX_LOCAL_TOPICS:
          local_topics.append(keywordsLocal[j][0])

   print("chunk ", i , " duration ", chunk.duration_seconds, " topics: ",local_topics )
   chunk_topic_list.append([i,chunk,local_topics,timestamps[i]])


100%|██████████| 8010/8010 [00:06<00:00, 1282.00frames/s]


chunk  0  duration  80.10498866213152  topics:  ['corb', 'bitcoin', 'podcast', 'listening']


100%|██████████| 1769/1769 [00:02<00:00, 638.53frames/s]


chunk  1  duration  17.697006802721088  topics:  ['music', 'london', 'corb', 'doing']


100%|██████████| 9321/9321 [00:09<00:00, 939.12frames/s] 


...

chunk  38  duration  39.69199546485261  topics:  ['jennings', 'musician', 'musicians', 'artist']


100%|██████████| 346/346 [00:00<00:00, 708.79frames/s]


chunk  39  duration  3.461995464852608  topics:  ['moon', 'right']


100%|██████████| 2829/2829 [00:01<00:00, 1466.11frames/s]

chunk  40  duration  28.29299319727891  topics:  ['bitcoin', 'wallet', 'podcast', 'satoshi']


In [91]:
# CAN BE DELETED
#printing results of chunk_topic_list.
combined_segments = []
for i,chunk,topics,timestamps in chunk_topic_list:
    print("chunk ", i , " duration ", chunk.duration_seconds, " topics: ",topics, " start:", timestamps[1], " end:", timestamps[2])

print(chunk_topic_list)


chunk  0  duration  80.10498866213152  topics:  ['corb', 'bitcoin', 'podcast', 'listening']  start: 0  end: 80
chunk  1  duration  17.697006802721088  topics:  ['music', 'london', 'corb', 'doing']  start: 80  end: 97
chunk  2  duration  93.21600907029479  topics:  ['music', 'radio', 'songwriting', 'underground']  start: 97  end: 191
chunk  3  duration  61.2409977324263  topics:  ['audience', 'culturally', 'diverse', 'music']  start: 191  end: 252
chunk  4  duration  14.63498866213152  topics:  ['canada', 'japan', 'australia', 'uk']  start: 252  end: 266
chunk  5  duration  156.40201814058958  topics:  ['music', 'listening', 'bluegrass', 'rock']  start: 266  end: 423
chunk  6  duration  6.458004535147392  topics:  ['democratic', 'fork', 'source', 'really']  start: 423  end: 429
chunk  7  duration  5.618004535147392  topics:  ['watch', 'interesting', 'going', 'think']  start: 429  end: 435
chunk  8  duration  39.71  topics:  ['bitcoin', 'cryptocurrencies', 'investing', 'future']  start: 

In [156]:
# if a chunk is shorter than 60 seconds and if the topics are the same, we combine the chunk with the previous and get a list of final audios called segments_list.
# also export the audio list to file with the name like 0-80-segment_0['corb', 'bitcoin', 'podcast', 'listening'].wav, where first numbers are the timestamps then the word segment and the number of the chunk. Then the list og topics.

#remove old segments in folder
mydir="./"
for f in os.listdir(mydir):
    if not f.startswith("time"):
        continue
    os.remove(f)

#find short chunks and combine with previous if is same topic
combined_segments = []
segment_list=[]
for i,chunk,topics,timestamp in chunk_topic_list:
    #print("chunk ", i , " duration ", chunk.duration_seconds, " topics: ",topics )
    if chunk.duration_seconds < 60:
        print(i, "shorter than 60 secs")
        # if i > 0 and i < len(segments) - 1:
        if 0 < i < len(chunk_topic_list) - 1:
            #if topics are in combined_segments[-1] topic list
            previous_topics = chunk_topic_list[i-1][2]
            print(i, "topics", topics)
            print(i, "previous topics ", previous_topics)
            topic_found = False
            for local_topic in topics:
                if local_topic in previous_topics:
                    topic_found = True

            if topic_found:
                print(i," same topic as previous")
                last_end = chunk_topic_list[i][3][1]
                last_topics = chunk_topic_list[i-1][2]
                print("last_end:",last_end)

                if i>17:
                    print(i)

                try:
                    combined_segments[-1] = [combined_segments[i-1][0] + chunk, chunk_topic_list[i-1][3][1], timestamp[2]] #+ chunk_topic_list[i+1]
                    # print("combining ", i ," with ", i-1, " start: ", timestamps[i][1], " ends ", timestamps[i][2], " start ",  timestamps[i-1][1], " end ",timestamps[i-1][2])
                except:
                    print("error renaming")


                start = combined_segments[0][1]

                end = timestamp[2]

                print("combining ", i ," with ", i-1, " start: ", start, " ends ", end)


                print("time{}-{}-segment_{}{}.wav".format(start,last_end,i-1,last_topics), " -> ", "time{}-{}-segment_{}{}.wav".format(start,end,i-1,(last_topics + topics)))

                # listing directories
                # print ("The dir is: %s"%os.listdir(os.getcwd()))

                try:
                    os.rename(("time{}-{}-segment_{}{}.wav".format(start,last_end,i-1,last_topics)),"time{}-{}-segment_{}{}.wav".format(start,end,i-1,(last_topics+topics)))
                except:
                    print("error renaming")

                for j, combi_seg in enumerate(combined_segments):
                    try:
                        print(j, " ",combi_seg[0].duration_seconds, " start: ", combi_seg[1], " end: ", combi_seg[2])
                    except:
                        print("error renaming")
            else:
                if i>17:
                    print(i)

                print(i,"NOT same topic as previous. Will Skip fragment.")
                segment_name = "time{}-{}-segment_{}{}.wav".format(timestamp[1],timestamp[2],i,topics)
                # chunk.export("segment_{}{}.wav".format(i,topics), format="wav")
                # chunk.export(segment_name, format="wav")
                print(segment_name, " duration ", chunk.duration_seconds)
                combined_segments = [chunk,timestamp[1],timestamp[2]]
                # combined_segments.append([chunk, timestamp[1],timestamp[2]])
        else:
            print(i, "is 0 or ", len(chunk_topic_list) )
            combined_segments.append([chunk, timestamp[1],timestamp[2]])
            print("combined_segments", i , " duration ", chunk.duration_seconds)
            segment_name = "time{}-{}-segment_{}{}.wav".format(timestamp[1],timestamp[2],i,topics)
            # chunk.export("segment_{}{}.wav".format(i,topics), format="wav")
            chunk.export(segment_name, format="wav")
            print(segment_name, " duration ", chunk.duration_seconds)
            segment_list.append([i,chunk,topics])
    else:
        # combined_segments.append(chunk)
        combined_segments.append([chunk, timestamp[1],timestamp[2]])
        segment_name = "time{}-{}-segment_{}{}.wav".format(timestamp[1],timestamp[2],i,topics)
        chunk.export(segment_name, format="wav")
        print(segment_name, " duration ", chunk.duration_seconds)
        segment_list.append([i,chunk,topics,segment_name])

        #limpiar comined_segments?
        #comined_segments = []


time0-80-segment_0['corb', 'bitcoin', 'podcast', 'listening'].wav  duration  80.10498866213152
1 shorter than 60 secs
1 topics ['music', 'london', 'corb', 'doing']
1 previous topics  ['corb', 'bitcoin', 'podcast', 'listening']
1  same topic as previous
last_end: 80
combining  1  with  0  start:  0  ends  97
time0-80-segment_0['corb', 'bitcoin', 'podcast', 'listening'].wav  ->  time0-97-segment_0['corb', 'bitcoin', 'podcast', 'listening', 'music', 'london', 'corb', 'doing'].wav
0   97.80199546485261  start:  0  end:  97
time97-191-segment_2['music', 'radio', 'songwriting', 'underground'].wav  duration  93.21600907029479
time191-252-segment_3['audience', 'culturally', 'diverse', 'music'].wav  duration  61.2409977324263
4 shorter than 60 secs
4 topics ['canada', 'japan', 'australia', 'uk']
4 previous topics  ['audience', 'culturally', 'diverse', 'music']
4 NOT same topic as previous. Will Skip fragment.
time252-266-segment_4['canada', 'japan', 'australia', 'uk'].wav  duration  14.63498866